In [1]:
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from retrievers import VectorStoreRetriever
from config import openai_key
from functions import devrev_functions

openai_embeddings = OpenAIEmbeddings(
    openai_api_key = openai_key
)

vs_retriever = VectorStoreRetriever(
    embeddings=openai_embeddings,
    name = "vs_rtr_01",
    init_functions=devrev_functions.copy()
)

In [2]:
vs_retriever.find_functions('what is my id')

[Document(page_content='{"name": "who_am_i", "description": "Returns the ID of the current user.", "parameters": {"type": "object", "properties": {}}}', metadata={'index': 8}),
 Document(page_content='{"name": "get_sprint_id", "description": "Returns the ID of the current sprint.", "parameters": {"type": "object", "properties": {}}}', metadata={'index': 4}),
 Document(page_content='{"name": "search_object_by_name", "description": "Given a search string, returns the ID of a matching object in the system of record. If multiple matches are found, it returns the one where the confidence is highest.", "parameters": {"type": "object", "properties": {"query": {"title": "query", "type": "string", "description": "The search string, could be for example customer\\u2019s name, part name, user name."}}, "required": []}}', metadata={'index': 6}),
 Document(page_content='{"name": "resolution_status", "description": "Gives an array of only unresolved issues", "parameters": {"type": "object", "propert

In [1]:
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from retrievers import CustomMultiQueryRetriever
from langchain.chat_models import ChatOpenAI
from config import openai_key
from functions import devrev_functions


hf_embeddings = HuggingFaceEmbeddings()

chat_llm = ChatOpenAI(
    openai_api_key = openai_key,
    temperature=0.7,
)

cmq_ret = CustomMultiQueryRetriever(
    chat_llm,
    embeddings=hf_embeddings,
    name = "cmq_ret_01",
    init_functions=devrev_functions.copy()
)

c:\Users\kayde\OneDrive\Desktop\devrev-openai-classic\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
cmq_ret.find_functions('find all my P0 issues')

[Document(page_content='{"name": "who_am_i", "description": "Returns the ID of the current user.", "parameters": {"type": "object", "properties": {}}}', metadata={'index': 8}),
 Document(page_content='{"name": "get_sprint_id", "description": "Returns the ID of the current sprint.", "parameters": {"type": "object", "properties": {}}}', metadata={'index': 4}),
 Document(page_content='{"name": "loop_condition", "description": "Executes a loop for a specified number of repetitions.", "parameters": {"type": "object", "properties": {"Repetitions": {"title": "Repetitions", "type": "integer", "description": "The number of repetitions for the loop."}, "Go_to": {"title": "Go_to", "type": "string", "description": "The target location or action to jump to after each iteration."}}, "required": []}}', metadata={'index': 19}),
 Document(page_content='{"name": "search_object_by_name", "description": "Given a search string, returns the ID of a matching object in the system of record. If multiple matche

In [6]:
from langchain.chat_models import ChatOpenAI

from verification import Verifier

chat_llm = ChatOpenAI(
    openai_api_key = openai_key
)

verifier = Verifier(
    chat_llm = chat_llm
)

In [24]:
query = "Summarize work items similar to don:core:dvrv-us-1:devo/0:issue/1"
schema = '''
[
    {
        "tool_name": "get_similar_work_items",
        "arguments": [
            {
                "argument_name": "work_id",
                "argument_value": "don:core:dvrv-us-1:devo/0:issue/1"
            }
        ]
    },
    {
        "tool_name": "summarize_objects",
        "arguments": [
            {
                "argument_name": "objects",
                "argument_value": "$$PREV[0]"
            }
        ]
    }
]'''
verifier(answer=schema,query=query,init_functions=devrev_functions)['text']



'False'

In [ ]:
from composers import ChainOfThoughtComposer
from examples import example_queries
composer = ChainOfThoughtComposer(
    chat_llm=chat_llm,
)